# Testing Sharadar Fundamentals - Stock Screening Examples

This notebook demonstrates how to use Sharadar SF1 fundamentals data in zipline-reloaded's Pipeline API to screen for quality stocks.

**What You'll Learn:**
- How to access 80+ fundamental metrics in Pipeline
- Point-in-time data handling (no look-ahead bias)
- Multiple stock screening strategies:
  - Magic Formula (Greenblatt)
  - High ROE companies
  - Profitable growth stocks
  - Value stocks (low P/E)
  - Quality companies (Piotroski F-Score)
- Data quality validation

**Prerequisites:**
- Sharadar bundle ingested with fundamentals
- See `docs/SHARADAR_FUNDAMENTALS_GUIDE.md` for setup

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from zipline.pipeline import Pipeline
from zipline.pipeline.data.sharadar import SharadarFundamentals
from zipline.pipeline.loaders.sharadar_fundamentals import make_sharadar_fundamentals_loader
from zipline.pipeline.engine import SimplePipelineEngine
from zipline.pipeline.domain import US_EQUITIES
from zipline.pipeline.factors import SimpleMovingAverage, Returns
from zipline.data.bundles import load, register
from zipline.data.bundles.sharadar_bundle import sharadar_bundle

# Set display options
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

# Set plot style
sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Imports complete")

## Load Bundle and Create Pipeline Engine

In [ ]:
# Register and load bundle
print("Loading Sharadar bundle...")
register('sharadar', sharadar_bundle())
bundle_data = load('sharadar')
print(f"✓ Bundle loaded")

# Create fundamentals loader
print("\nCreating fundamentals loader...")
fundamentals_loader = make_sharadar_fundamentals_loader('sharadar')
print(f"✓ Loader created: {fundamentals_loader.fundamentals_path}")

# Find a recent trading session
trading_calendar = bundle_data.equity_daily_bar_reader.trading_calendar
sessions = trading_calendar.sessions_in_range(
    pd.Timestamp('2024-11-01'),
    pd.Timestamp('2024-11-15')
)
test_date = sessions[-1]
print(f"\n✓ Using test date: {test_date.date()}")

# Create Pipeline engine
engine = SimplePipelineEngine(
    get_loader=lambda column: fundamentals_loader,
    asset_finder=bundle_data.asset_finder,
    default_domain=US_EQUITIES,
)
print("✓ Pipeline engine ready")

## Explore Available Fundamentals Data

First, let's see what data is available and check data coverage.

In [ ]:
# Create a pipeline with key fundamental metrics
explore_pipeline = Pipeline(
    columns={
        # Income Statement
        'revenue': SharadarFundamentals.revenue.latest,
        'netinc': SharadarFundamentals.netinc.latest,
        'ebitda': SharadarFundamentals.ebitda.latest,
        'eps': SharadarFundamentals.eps.latest,
        
        # Balance Sheet
        'assets': SharadarFundamentals.assets.latest,
        'equity': SharadarFundamentals.equity.latest,
        'debt': SharadarFundamentals.debt.latest,
        'cashnequsd': SharadarFundamentals.cashnequsd.latest,
        
        # Ratios
        'roe': SharadarFundamentals.roe.latest,
        'roa': SharadarFundamentals.roa.latest,
        'pe': SharadarFundamentals.pe.latest,
        'pb': SharadarFundamentals.pb.latest,
        'de': SharadarFundamentals.de.latest,
        
        # Market Data
        'marketcap': SharadarFundamentals.marketcap.latest,
        'ev': SharadarFundamentals.ev.latest,
    },
)

# Run pipeline
print("Running exploratory pipeline...")
explore_data = engine.run_pipeline(explore_pipeline, test_date, test_date)

print(f"\n{'='*60}")
print("Data Coverage Summary")
print(f"{'='*60}")
print(f"Total assets in universe: {len(explore_data):,}")
print(f"Assets with any data: {explore_data.dropna(how='all').shape[0]:,}")
print(f"\nData coverage by metric:")

coverage = pd.DataFrame({
    'Metric': explore_data.columns,
    'Non-Null': [explore_data[col].notna().sum() for col in explore_data.columns],
    'Coverage %': [(explore_data[col].notna().sum() / len(explore_data) * 100) for col in explore_data.columns]
}).sort_values('Non-Null', ascending=False)

print(coverage.to_string(index=False))

## Data Quality Check - Sample Companies

In [ ]:
# Show data for well-known large-cap companies
print("Sample data for major companies:\n")

# Get companies with complete data and high market cap
complete_data = explore_data.dropna(subset=['revenue', 'netinc', 'assets', 'marketcap'])
large_caps = complete_data.nlargest(20, 'marketcap')

# Display key metrics
display_cols = ['revenue', 'netinc', 'eps', 'roe', 'pe', 'marketcap']
print(large_caps[display_cols].to_string())

print(f"\n✓ Showing top 20 companies by market cap with complete fundamental data")

## Strategy 1: Magic Formula (Greenblatt)

The Magic Formula combines:
- **Earnings Yield**: EBIT / Enterprise Value (prefer high)
- **Return on Capital**: EBIT / (Net Working Capital + Net Fixed Assets) (prefer high)

For simplicity, we'll use ROE and inverse P/E as proxies.

In [ ]:
# Filter for companies with positive earnings and reasonable valuation
magic_formula_data = explore_data[
    (explore_data['netinc'] > 0) &
    (explore_data['roe'] > 0) &
    (explore_data['pe'] > 0) &
    (explore_data['pe'] < 50) &
    (explore_data['marketcap'] > 1e9)  # Market cap > $1B
].copy()

# Calculate ranks
magic_formula_data['roe_rank'] = magic_formula_data['roe'].rank(ascending=False)
magic_formula_data['earnings_yield'] = 1 / magic_formula_data['pe']
magic_formula_data['yield_rank'] = magic_formula_data['earnings_yield'].rank(ascending=False)
magic_formula_data['combined_rank'] = magic_formula_data['roe_rank'] + magic_formula_data['yield_rank']

# Top 20 by Magic Formula
top_magic = magic_formula_data.nsmallest(20, 'combined_rank')

print("="*80)
print("Top 20 Stocks - Magic Formula (ROE + Earnings Yield)")
print("="*80)
print(top_magic[['revenue', 'netinc', 'roe', 'pe', 'earnings_yield', 'combined_rank']].to_string())

print(f"\nFiltered from {len(explore_data):,} down to {len(magic_formula_data):,} qualifying stocks")

## Strategy 2: High ROE Quality Companies

Screen for:
- ROE > 20% (exceptional profitability)
- Positive net income
- Reasonable debt levels (D/E < 2)

In [ ]:
high_roe_stocks = explore_data[
    (explore_data['roe'] > 0.20) &
    (explore_data['netinc'] > 0) &
    (explore_data['de'] < 2) &
    (explore_data['marketcap'] > 5e8)  # Market cap > $500M
].copy()

# Sort by ROE
high_roe_stocks = high_roe_stocks.sort_values('roe', ascending=False)

print("="*80)
print("Top 20 High ROE Companies (ROE > 20%, Low Debt)")
print("="*80)
print(high_roe_stocks[['revenue', 'netinc', 'roe', 'de', 'pe', 'marketcap']].head(20).to_string())

print(f"\n✓ Found {len(high_roe_stocks)} stocks with ROE > 20%")

## Strategy 3: Profitable Growth Stocks

Screen for:
- Positive revenue and net income
- High revenue (>$1B)
- Strong profit margin
- Reasonable P/E ratio

In [ ]:
# Calculate profit margin
growth_data = explore_data[
    (explore_data['revenue'] > 0) &
    (explore_data['netinc'] > 0)
].copy()

growth_data['profit_margin'] = growth_data['netinc'] / growth_data['revenue']

# Filter for profitable growth
growth_stocks = growth_data[
    (growth_data['revenue'] > 1e9) &  # Revenue > $1B
    (growth_data['profit_margin'] > 0.10) &  # Profit margin > 10%
    (growth_data['pe'] > 0) &
    (growth_data['pe'] < 30)
].copy()

# Sort by profit margin
growth_stocks = growth_stocks.sort_values('profit_margin', ascending=False)

print("="*80)
print("Top 20 Profitable Growth Stocks (Revenue > $1B, Margin > 10%, P/E < 30)")
print("="*80)
print(growth_stocks[['revenue', 'netinc', 'profit_margin', 'pe', 'marketcap']].head(20).to_string())

print(f"\n✓ Found {len(growth_stocks)} profitable growth stocks")

## Strategy 4: Deep Value Stocks

Screen for:
- Low P/E ratio (< 10)
- Low P/B ratio (< 1.5)
- Positive earnings
- Not overleveraged

In [ ]:
value_stocks = explore_data[
    (explore_data['pe'] > 0) &
    (explore_data['pe'] < 10) &
    (explore_data['pb'] > 0) &
    (explore_data['pb'] < 1.5) &
    (explore_data['netinc'] > 0) &
    (explore_data['de'] < 1.5) &
    (explore_data['marketcap'] > 2e8)  # Market cap > $200M
].copy()

# Sort by P/E
value_stocks = value_stocks.sort_values('pe')

print("="*80)
print("Top 20 Deep Value Stocks (P/E < 10, P/B < 1.5, Profitable)")
print("="*80)
print(value_stocks[['revenue', 'netinc', 'pe', 'pb', 'de', 'marketcap']].head(20).to_string())

print(f"\n✓ Found {len(value_stocks)} deep value stocks")

## Strategy 5: Quality Score (Simplified Piotroski F-Score)

Create a quality score based on:
- Profitability (positive net income)
- Strong ROE (> 10%)
- Strong ROA (> 5%)
- Low debt (D/E < 1)
- Reasonable valuation (P/E < 25)

In [ ]:
# Create quality scoring dataset
quality_data = explore_data[
    (explore_data['netinc'].notna()) &
    (explore_data['roe'].notna()) &
    (explore_data['roa'].notna()) &
    (explore_data['de'].notna()) &
    (explore_data['pe'].notna()) &
    (explore_data['marketcap'] > 5e8)
].copy()

# Calculate quality score (0-5 points)
quality_data['quality_score'] = 0
quality_data.loc[quality_data['netinc'] > 0, 'quality_score'] += 1
quality_data.loc[quality_data['roe'] > 0.10, 'quality_score'] += 1
quality_data.loc[quality_data['roa'] > 0.05, 'quality_score'] += 1
quality_data.loc[quality_data['de'] < 1.0, 'quality_score'] += 1
quality_data.loc[(quality_data['pe'] > 0) & (quality_data['pe'] < 25), 'quality_score'] += 1

# Get high quality stocks (score >= 4)
high_quality = quality_data[quality_data['quality_score'] >= 4].sort_values('quality_score', ascending=False)

print("="*80)
print("Top 20 High Quality Stocks (Quality Score >= 4/5)")
print("="*80)
print(high_quality[['revenue', 'netinc', 'roe', 'roa', 'de', 'pe', 'quality_score']].head(20).to_string())

print(f"\n✓ Found {len(high_quality)} high quality stocks (score >= 4)")

# Show score distribution
print("\nQuality Score Distribution:")
print(quality_data['quality_score'].value_counts().sort_index(ascending=False))

## Visualization: ROE vs P/E Scatter Plot

Visualize the relationship between profitability (ROE) and valuation (P/E).

In [ ]:
# Prepare data for plotting
plot_data = explore_data[
    (explore_data['roe'] > 0) &
    (explore_data['roe'] < 1) &  # Cap at 100% ROE
    (explore_data['pe'] > 0) &
    (explore_data['pe'] < 50) &
    (explore_data['marketcap'] > 1e9)
].copy()

# Create scatter plot
plt.figure(figsize=(14, 8))
scatter = plt.scatter(
    plot_data['roe'] * 100,  # Convert to percentage
    plot_data['pe'],
    c=np.log10(plot_data['marketcap']),
    s=50,
    alpha=0.6,
    cmap='viridis'
)

plt.xlabel('ROE (%)', fontsize=12)
plt.ylabel('P/E Ratio', fontsize=12)
plt.title('ROE vs P/E Ratio (Market Cap > $1B)', fontsize=14, fontweight='bold')
plt.colorbar(scatter, label='log10(Market Cap)')
plt.grid(True, alpha=0.3)

# Add quadrant lines
plt.axvline(x=15, color='red', linestyle='--', alpha=0.5, label='ROE = 15%')
plt.axhline(y=20, color='blue', linestyle='--', alpha=0.5, label='P/E = 20')
plt.legend()

# Annotate sweet spot
plt.text(25, 10, 'High ROE\nLow P/E\n(Sweet Spot)', 
         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.3),
         fontsize=10, ha='center')

plt.tight_layout()
plt.show()

print(f"\n✓ Plotted {len(plot_data)} stocks")

## Data Quality Validation

Check for data consistency and potential issues.

In [ ]:
print("="*80)
print("Data Quality Checks")
print("="*80)

validation_data = explore_data.copy()

# 1. Check for negative equity
negative_equity = validation_data[validation_data['equity'] < 0]
print(f"\n1. Companies with negative equity: {len(negative_equity):,}")
if len(negative_equity) > 0:
    print("   (This is normal for distressed companies or those with accumulated losses)")

# 2. Check for extreme P/E ratios
extreme_pe = validation_data[(validation_data['pe'] > 100) | (validation_data['pe'] < -50)]
print(f"\n2. Companies with extreme P/E (|P/E| > 100): {len(extreme_pe):,}")
if len(extreme_pe) > 0:
    print("   (Extreme P/E values often indicate low/negative earnings)")

# 3. Check for missing key metrics
missing_revenue = validation_data[validation_data['revenue'].isna()]
print(f"\n3. Companies missing revenue data: {len(missing_revenue):,}")

# 4. Check debt/equity ratio consistency
has_de = validation_data[validation_data['de'].notna()]
high_leverage = has_de[has_de['de'] > 5]
print(f"\n4. Companies with D/E > 5 (highly leveraged): {len(high_leverage):,}")
if len(high_leverage) > 0:
    print("   (High leverage companies - financials or distressed)")

# 5. Summary statistics
print("\n5. Summary Statistics for Key Ratios:\n")
ratio_cols = ['roe', 'roa', 'pe', 'pb', 'de']
print(validation_data[ratio_cols].describe().to_string())

print("\n" + "="*80)
print("✓ Data quality validation complete")
print("="*80)

## Export Top Picks for Further Analysis

In [ ]:
# Combine top picks from different strategies
top_picks = pd.DataFrame()

if len(top_magic) > 0:
    top_picks = pd.concat([
        top_picks,
        top_magic.head(10).assign(strategy='Magic Formula')
    ])

if len(high_roe_stocks) > 0:
    top_picks = pd.concat([
        top_picks,
        high_roe_stocks.head(10).assign(strategy='High ROE')
    ])

if len(value_stocks) > 0:
    top_picks = pd.concat([
        top_picks,
        value_stocks.head(10).assign(strategy='Deep Value')
    ])

if len(high_quality) > 0:
    top_picks = pd.concat([
        top_picks,
        high_quality.head(10).assign(strategy='Quality')
    ])

# Remove duplicates (some stocks may appear in multiple strategies)
print(f"Total picks (with duplicates): {len(top_picks)}")
print(f"Unique stocks: {top_picks.index.nunique()}\n")

# Show stocks that appear in multiple strategies
strategy_counts = top_picks.groupby(top_picks.index)['strategy'].apply(list).to_frame()
multi_strategy = strategy_counts[strategy_counts['strategy'].apply(len) > 1]

if len(multi_strategy) > 0:
    print("="*80)
    print("Stocks appearing in MULTIPLE strategies (highest conviction):")
    print("="*80)
    for idx, row in multi_strategy.iterrows():
        print(f"\n{idx}:")
        print(f"  Strategies: {', '.join(row['strategy'])}")
        metrics = explore_data.loc[idx, ['revenue', 'netinc', 'roe', 'pe', 'marketcap']]
        print(f"  Revenue: ${metrics['revenue']:,.0f}")
        print(f"  Net Income: ${metrics['netinc']:,.0f}")
        print(f"  ROE: {metrics['roe']:.1%}")
        print(f"  P/E: {metrics['pe']:.1f}")
        print(f"  Market Cap: ${metrics['marketcap']:,.0f}")
else:
    print("No stocks appear in multiple strategies.")

print("\n✓ Analysis complete!")

## Summary

This notebook demonstrated:

1. ✅ **Data Access**: Successfully loaded 15+ fundamental metrics from Sharadar SF1
2. ✅ **Point-in-Time**: All data is as-of the test date (no look-ahead bias)
3. ✅ **Multiple Strategies**: Implemented 5 different screening approaches
4. ✅ **Data Quality**: Validated data consistency and coverage
5. ✅ **Actionable Results**: Identified top stocks for further research

### Next Steps

1. **Backtest Strategies**: Use these screens in a full zipline backtest
2. **Add More Metrics**: Explore the 80+ available fundamental columns
3. **Time Series Analysis**: Look at fundamental trends over time
4. **Combine with Technicals**: Add price/volume factors to screens

### Additional Resources

- Full fundamentals guide: `docs/SHARADAR_FUNDAMENTALS_GUIDE.md`
- Available metrics: `src/zipline/pipeline/data/sharadar.py`
- Example strategies: See guide for complete backtest examples

---

**Author**: Kamran Sokhanvari @ Hidden Point Capital  
**Repository**: https://github.com/ksokhanvari/zipline-reloaded